**IMPORT REQUIRED LIBRARIES**

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import cv2
import pathlib
import glob
import shutil
import os
import random

**CONNECTING TO WANDB**

In [2]:
# #---------------------------------------install and import wandb -------------------------------------------------
!pip install wandb -qqq
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 10.8MB/s 
     |████████████████████████████████| 163kB 38.4MB/s 
     |████████████████████████████████| 133kB 37.1MB/s 
     |████████████████████████████████| 102kB 9.3MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 


In [3]:
# #--------------------------------------------login to wandb --------------------------------------------
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
#--------------------------------caution: terminal commands ---------------------------------------------
%cd
%cd .keras/datasets/
!rm -r *

/root
[Errno 2] No such file or directory: '.keras/datasets/'
/root
rm: cannot remove '*': No such file or directory


In [5]:
########################################### download data from given url ###############################################

dataset_url = "https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"
data_dir = tf.keras.utils.get_file('dakshina_dataset_v1.0', origin=dataset_url, untar=True)

2008342528/2008340480 [==============================] - 19s 0us/step


In [6]:
#----------------------------------terminal command -----------------------------------------------
%cd /root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons/
!ls

/root/.keras/datasets/dakshina_dataset_v1.0/hi/lexicons
hi.translit.sampled.dev.tsv   hi.translit.sampled.train.tsv
hi.translit.sampled.test.tsv


In [7]:
train_data_path = "hi.translit.sampled.train.tsv"
test_data_path = "hi.translit.sampled.test.tsv"
validation_data_path = "hi.translit.sampled.dev.tsv"

**UTILITY FUNCTIONS FOR PREPOCESSING**

In [8]:
################################# function for vectorizing the data ##########################################

def vectorize_data(train_data_path):
  input_texts = []
  target_texts = []
  input_characters = set()
  target_characters = set()
  with open(train_data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
  for line in lines[: min(num_samples, len(lines) - 1)]:
    target_text, input_text, _ = line.split("\t")
    #---------------We use "tab" as the "start sequence" character---------------------
    #----------------for the targets, and "\n" as "end sequence" character-----------------.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

  #--------------------------------artificially added-----------------------------
  input_characters.add(" ")
  target_characters.add(" ")

  input_characters = sorted(list(input_characters))
  target_characters = sorted(list(target_characters))

  num_encoder_tokens = len(input_characters)
  num_decoder_tokens = len(target_characters)
  max_encoder_seq_length = max([len(txt) for txt in input_texts])
  max_decoder_seq_length = max([len(txt) for txt in target_texts])

  print("Number of samples:", len(input_texts))
  print("Number of unique input tokens:", num_encoder_tokens)
  print("Number of unique output tokens:", num_decoder_tokens)
  print("Max sequence length for inputs:", max_encoder_seq_length)
  print("Max sequence length for outputs:", max_decoder_seq_length)

  input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
  target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  input_details = [input_characters, input_texts, input_token_index, num_encoder_tokens, max_encoder_seq_length]
  target_details = [target_characters, target_texts, target_token_index, num_decoder_tokens, max_decoder_seq_length]

  return (input_details, target_details)

In [9]:
################### function for converting the data into apropriate ONE-Hot vector ######################

def onehot(input_details, target_details):

    #---------------------------unzipping information-----------------------------------
    input_characters = input_details[0]
    input_texts = input_details[1]
    input_token_index = input_details[2]
    num_encoder_tokens = input_details[3]
    max_encoder_seq_length = input_details[4]

    target_characters = target_details[0]
    target_texts = target_details[1]
    target_token_index = target_details[2]
    num_decoder_tokens = target_details[3]
    max_decoder_seq_length = target_details[4]

    #---------------------------- creating 3-Dim  matrics with all entries = 0 ----------------------------------- 
    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
    decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0

        encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
        for t, char in enumerate(target_text):
# --------------decoder_target_data is ahead of decoder_input_data by one timestep ----------------------------
            decoder_input_data[i, t, target_token_index[char]] = 1.0
            
            if t > 0:
# ----------------decoder_target_data will be ahead by one timestep----------------------------------
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

        decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
        decoder_target_data[i, t:, target_token_index[" "]] = 1.0
    
    return (encoder_input_data, decoder_input_data, decoder_target_data)

In [10]:
################### function for creating data into appropriate form required for embedding ###################

def get_input_for_embedding(input_details, embed_size, train_details = None):

  #---------------------------unzipping information-----------------------------------
  # print("\n input_details \n",input_details)
  input_characters = input_details[0]
  input_texts = input_details[1]
  input_token_index = input_details[2]
  num_encoder_tokens = input_details[3]
  max_encoder_seq_length = input_details[4]

  if train_details != None:
    # print("validation")
    input_token_index = train_details[2]
  
  input_array = np.zeros((len(input_texts), max_encoder_seq_length), dtype="float32")
  for (i, input_text) in enumerate(input_texts):
        for (t, char) in enumerate(input_text):
          input_array[i, t] = input_token_index[" "]
          if char in input_token_index:
            input_array[i, t] = input_token_index[char]
        input_array[i, t + 1 :] = input_token_index[" "]

  # print("input array",input_array)
  return input_array



**MACHINE TRANSLITERATOR**


In [11]:
class Machine_Transliterator():

  ############################################# constructor for class Machine_Transliterator ##########################################

  def __init__(self,max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, decoder_embed_size,
               num_encoder_layers,num_decoder_layers,epochs, hidden_layer_size,
               num_encoder_tokens, cell_type, num_decoder_tokens,input_token_index, target_token_index, 
               activation="softmax",optimizer="rmsprop",dropout=0.5):
    
    self.cell_type= cell_type
    self.hidden_layer_size = hidden_layer_size  
    self.optimizer = optimizer
    self.activation = activation   
    self.dropout=dropout

    #-------------------------------------- Number of hidden layers -------------------------------------

    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers=num_decoder_layers

    #-------------------------------------- sequence length -------------------------------------
    self.max_decoder_seq_length=max_decoder_seq_length
    self.max_encoder_seq_length=max_encoder_seq_length

    #---------------------------------------------Embedding size-------------------------------------
    self.encoder_embed_size = encoder_embed_size
    self.decoder_embed_size = decoder_embed_size
    
    #-----------------information obtained after preprocessing of data-------------------------------------
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens

    #-----------------------------dictionaries----------------------------------------------------
    self.input_token_index = input_token_index
    self.target_token_index = target_token_index

 


#########################################function to build model ###########################################

  def build_model(self):

    
    encoder_inputs = keras.Input(shape=(None,))   
    encoder_embedding_output = tf.keras.layers.Embedding(self.num_encoder_tokens, self.encoder_embed_size)(encoder_inputs)


#------------------------------ if cell type = LSTM -------------------------------------------------------------
    if self.cell_type == "lstm":
     #--------------------- encoder -----------------------------------
      encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
      encoder_outputs, state_h, state_c = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        encoder_outputs, state_h, state_c = encoder(encoder_outputs)
      encoder_states = [state_h, state_c]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(None, ))      
      decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)

      decoder = keras.layers.LSTM(self.hidden_layer_size, return_sequences=True, return_state=True,dropout=self.dropout,use_bias=True)
      decoder_outputs, _, _= decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.LSTM(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        decoder_outputs, _ , _= decoder(decoder_outputs, initial_state = encoder_states)



#------------------------------ if cell type = Simple RNN -------------------------------------------------------------
    elif self.cell_type == "rnn":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout, use_bias=True)
        encoder_outputs, state = encoder(encoder_outputs)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(None,))      
      decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)

      decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.SimpleRNN(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout, use_bias=True)
        decoder_outputs, _= decoder(decoder_outputs, initial_state = encoder_states)



#------------------------------ if cell type = GRU -------------------------------------------------------------
    elif self.cell_type == "gru":
      #--------------------- encoder -----------------------------------
      encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
      encoder_outputs, state = encoder(encoder_embedding_output)
      for i in range(1,self.num_encoder_layers):
        encoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        encoder_outputs, state = encoder(encoder_outputs)
      encoder_states = [state]

      #---------------------------decoder ---------------------------------------------------
      decoder_inputs = keras.Input(shape=(None, ))      
      decoder_embedding_output = tf.keras.layers.Embedding(self.num_decoder_tokens, self.decoder_embed_size)(decoder_inputs)
      
      decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True, dropout=self.dropout,use_bias=True)
      decoder_outputs, _ = decoder(decoder_embedding_output, initial_state = encoder_states)
      for i in range(1,self.num_decoder_layers):
        decoder = keras.layers.GRU(self.hidden_layer_size, return_state=True,return_sequences=True,dropout=self.dropout,use_bias=True)
        decoder_outputs, _ = decoder(decoder_outputs, initial_state = encoder_states)


    decoder_dense = keras.layers.Dense(self.num_decoder_tokens, activation = self.activation,use_bias=True)
    decoder_outputs = decoder_dense(decoder_outputs)
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
         #-----------------compile the model -------------------------------------
    model.compile(
         optimizer=self.optimizer,
         loss="categorical_crossentropy",
         metrics=["accuracy"]
         ) 

#-------------------------- return final model ---------------------------------------------------------
    return model





#########################################function for training the model ###########################################

  def train_model(self,encoder_input_data,decoder_input_data,decoder_target_data,epochs,batch_size,
                  val_encoder_input_data, val_decoder_input_data, val_decoder_target_data):
    
     model=self.build_model()
  
     model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size = batch_size,
        epochs = epochs,
        validation_data = ([val_encoder_input_data, val_decoder_input_data],val_decoder_target_data), 
        callbacks=[WandbCallback()]
        )
     return model

#===================================== end of class Machine_Transliterator ==========================================



**PARAMETERS**

In [12]:
cell_type = "gru" # Type of the recurring unit
batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
hidden_layer_size= 512  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
activation = "softmax" #activation
optimizer = "Adam"  #optimizer
encoder_embed_size = 32 #Encoder embedsize
decoder_embed_size = 32 #Decoder embedsize
num_encoder_layers=3  # number of hidden layers in encoder
num_decoder_layers=3   # number of hidden layers in decoder
dropout=0.05 #Dropout

**PREPROCESSING THE DATA**

In [13]:
############################# preprocessing the train data ################################

#---------------------------- vectorizing train data ----------------------------------
input_details, target_details = vectorize_data(train_data_path)

#------------------------converting data into one-hot representation ---------------------------
encoder_input_data, decoder_input_data, decoder_target_data = onehot(input_details, target_details)

#------------------------------- unzipping the data ----------------------------------------
input_characters = input_details[0]
input_texts = input_details[1]
input_token_index = input_details[2]
num_encoder_tokens = input_details[3]
max_encoder_seq_length = input_details[4]
target_characters = target_details[0]
target_texts = target_details[1]
target_token_index = target_details[2]
num_decoder_tokens = target_details[3]
max_decoder_seq_length = target_details[4]

#-------------------- converting input data into appropriate embedding ----------------------
encoder_input_data = get_input_for_embedding(input_details, encoder_embed_size)
decoder_input_data = get_input_for_embedding(target_details, decoder_embed_size)

Number of samples: 10000
Number of unique input tokens: 27
Number of unique output tokens: 64
Max sequence length for inputs: 18
Max sequence length for outputs: 20


In [14]:
############################# preprocessing the validation data ################################

#---------------------------- vectorizing validation data ----------------------------------
val_input_details, val_target_details = vectorize_data(validation_data_path)

#------------------------converting data into one-hot representation ---------------------------
val_encoder_input_data, val_decoder_input_data, val_decoder_target_data = onehot(val_input_details, val_target_details)

#------------------------------- unzipping the data ----------------------------------------
val_input_characters = val_input_details[0]
val_input_texts = val_input_details[1]
val_input_token_index = val_input_details[2]
val_num_encoder_tokens = val_input_details[3]
val_max_encoder_seq_length = val_input_details[4]
val_target_characters = val_target_details[0]
val_target_texts = val_target_details[1]
val_target_token_index = val_target_details[2]
val_num_decoder_tokens = val_target_details[3]
val_max_decoder_seq_length = val_target_details[4]

#-------------------- converting input data into appropriate embedding ----------------------
val_encoder_input_data = get_input_for_embedding(val_input_details, encoder_embed_size, input_details)
val_decoder_input_data = get_input_for_embedding(val_target_details, decoder_embed_size, target_details)

Number of samples: 4358
Number of unique input tokens: 27
Number of unique output tokens: 64
Max sequence length for inputs: 18
Max sequence length for outputs: 16


**CREATING MACHINE TRANSLITERATOR**

In [15]:
# ########################### creating machine transliterator object ###############################
# machine = Machine_Transliterator(
#     max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
#     decoder_embed_size,num_encoder_layers,num_decoder_layers,
#     batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
#      input_token_index,target_token_index, activation, optimizer,dropout
#     )

**TRAINING**



In [16]:
# model = machine.train_model(
#     encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
#     val_encoder_input_data, val_decoder_input_data, val_decoder_target_data
#     )

In [17]:
# tf.keras.utils.plot_model(model,to_file='model.png',show_shapes=True)

In [18]:
cell_type = ["lstm","rnn","gru"]
batch_size = [16,32,64,128]  # Batch size for training.
epochs = [10,20,30,50]  # Number of epochs to train for.
hidden_layer_size= [128,256,512]  # Latent dimensionality of the encoding space.
activation = "softmax"
optimizer = "Adam"
num_hidden_layers=[2,5,10]
encoder_embed_size = [27,64] #Encoder embedsize
decoder_embed_size = [27,64] #Decoder embedsize
num_encoder_layers=[1,2,3,4]  # number of hidden layers in encoder
num_decoder_layers=[1,2,3,4]    # number of hidden layers in decoder
dropout=[0.00,0.01, 0.5,0.0001] #Dropout

In [19]:
sweep_config={
              "method":"random",
              'metric' : {
                            'name' : 'val_accuracy',
                            'goal' : 'maximize',
                         },
          "parameters" : {
                            "cell_type":{"values":["lstm","rnn", "gru"]},
                            "batch_size":{"values": [32,64,128]},
                            "epochs":{"values":[40,60]}, 
                            "hidden_layer_size":{"values": [128,256,512,1024]}, 
                            "num_hidden_layers": {"values": [2,5,10]},
                            "encoder_embed_size": {"values": [27,64] },
                            "decoder_embed_size": {"values":[27,64]  },
                            "num_encoder_layers": {"values": [1,2,3,4]},
                            "num_decoder_layers":{"values":[1,2,3,4] },
                            "dropout": {"values":[0.00,0.01, 0.5,0.0001]}
                         }
              }
sweep_id = wandb.sweep(sweep_config, project="Machine_Transliterator")

Create sweep with ID: malfnxb9
Sweep URL: https://wandb.ai/anshikag_2210/Machine_Transliterator/sweeps/malfnxb9


In [20]:
def run():

  wb = wandb.init()
  config = wb.config

  #----------------------sweep parameters------------------------------------
  cell_type =config.cell_type 
  batch_size = config.batch_size 
  epochs = config.epochs 
  hidden_layer_size= config.hidden_layer_size
  encoder_embed_size =config.encoder_embed_size
  decoder_embed_size = config.decoder_embed_size
  num_encoder_layers=config.num_encoder_layers
  num_decoder_layers= config.num_decoder_layers 
  dropout=config.dropout
  
  ########################### creating machine transliterator object ###############################
  machine = Machine_Transliterator(
      max_encoder_seq_length,max_decoder_seq_length,encoder_embed_size, 
      decoder_embed_size,num_encoder_layers,num_decoder_layers,
      batch_size, hidden_layer_size, num_encoder_tokens, cell_type, num_decoder_tokens, 
      input_token_index,target_token_index, activation, optimizer,dropout
      )
  model = machine.train_model(
      encoder_input_data, decoder_input_data,decoder_target_data,epochs,batch_size,
      val_encoder_input_data, val_decoder_input_data, val_decoder_target_data
      )  
  return

In [ ]:
wandb.agent(sweep_id, run)

wandb: Agent Starting Run: 1bt0t9pc with config:
wandb: 	batch_size: 64
wandb: 	cell_type: gru
wandb: 	decoder_embed_size: 27
wandb: 	dropout: 0
wandb: 	encoder_embed_size: 64
wandb: 	epochs: 60
wandb: 	hidden_layer_size: 512
wandb: 	num_decoder_layers: 2
wandb: 	num_encoder_layers: 2
wandb: 	num_hidden_layers: 10
wandb: Currently logged in as: anshikag_2210 (use `wandb login --relogin` to force relogin)


Epoch 1/60
157/157 [==============================] - 47s 74ms/step - loss: 1.7397 - accuracy: 0.6353 - val_loss: 1.5222 - val_accuracy: 0.6549
Epoch 2/60
157/157 [==============================] - 10s 63ms/step - loss: 0.9760 - accuracy: 0.7299 - val_loss: 1.4463 - val_accuracy: 0.6750
Epoch 3/60
157/157 [==============================] - 10s 62ms/step - loss: 0.9005 - accuracy: 0.7484 - val_loss: 1.4263 - val_accuracy: 0.6838
Epoch 4/60
157/157 [==============================] - 10s 62ms/step - loss: 0.8333 - accuracy: 0.7631 - val_loss: 1.4271 - val_accuracy: 0.6870
Epoch 5/60
157/157 [==============================] - 10s 62ms/step - loss: 0.7896 - accuracy: 0.7760 - val_loss: 1.4402 - val_accuracy: 0.6963
Epoch 6/60
157/157 [==============================] - 10s 62ms/step - loss: 0.7164 - accuracy: 0.7937 - val_loss: 1.4305 - val_accuracy: 0.7016
Epoch 7/60
157/157 [==============================] - 10s 62ms/step - loss: 0.6193 - accuracy: 0.8184 - val_loss: 1.3463 - val_accuracy:

epoch,59
loss,0.02215
accuracy,0.99287
val_loss,1.52724
val_accuracy,0.78839
_runtime,633
_timestamp,1619527767
_step,59
best_val_loss,1.14417
best_epoch,10


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▆▅▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▂▃▃▄▅▆▇▇███████████████████████████████
val_loss,█▇▆▆▅▅▂▁▁▁▂▂▃▃▄▄▄▅▅▅▅▅▅▇▆▆▆▇▇▇▇▇▆▇▇▇██▇█
val_accuracy,▁▂▃▃▄▄▆▇▇█▇▇██▇█▇█▇████▇█▇██████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: j1e5mtdv with config:
wandb: 	batch_size: 64
wandb: 	cell_type: gru
wandb: 	decoder_embed_size: 64
wandb: 	dropout: 0.01
wandb: 	encoder_embed_size: 64
wandb: 	epochs: 40
wandb: 	hidden_layer_size: 1024
wandb: 	num_decoder_layers: 3
wandb: 	num_encoder_layers: 4
wandb: 	num_hidden_layers: 10


Epoch 1/40
157/157 [==============================] - 53s 265ms/step - loss: 1.8545 - accuracy: 0.6451 - val_loss: 1.4684 - val_accuracy: 0.6584
Epoch 2/40
157/157 [==============================] - 39s 251ms/step - loss: 0.9766 - accuracy: 0.7283 - val_loss: 1.4019 - val_accuracy: 0.6723
Epoch 3/40
157/157 [==============================] - 39s 250ms/step - loss: 0.8764 - accuracy: 0.7489 - val_loss: 1.3679 - val_accuracy: 0.6852
Epoch 4/40
 68/157 [===========>..................] - ETA: 19s - loss: 0.7847 - accuracy: 0.7700